In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip

--2023-04-21 19:53:14--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.132.134
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.132.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895569552 (854M) [binary/octet-stream]
Saving to: ‘datasetb2d9982.zip’

datasetb2d9982.zip  100%[===================>] 854.08M  95.4MB/s    in 8.9s    

2023-04-21 19:53:23 (95.6 MB/s) - ‘datasetb2d9982.zip’ saved [895569552/895569552]



In [ ]:
!unzip /content/datasetb2d9982.zip -d /content/

Archive:  /content/datasetb2d9982.zip
   creating: /content/dataset/
  inflating: /content/dataset/sample_submission.csv  
  inflating: /content/dataset/train.csv  
  inflating: /content/dataset/test.csv  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/dataset/train.csv')

In [ ]:
data.shape

(2249698, 6)

In [ ]:
data.isna().sum()

PRODUCT_ID               0
TITLE                   12
BULLET_POINTS       837364
DESCRIPTION        1157381
PRODUCT_TYPE_ID          0
PRODUCT_LENGTH           0
dtype: int64

In [ ]:
data_less = data.sample(30000).reset_index(drop=True)
data_less.head(10)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,2522839,Children's Joy Figure Soft Stuffed Plush Kids ...,"[Fibre Filling,Washable,Non-Toxic,Skin Friendl...",Children's joy Eco Friendly Imported Plush Sof...,1117,787.401574
1,1724885,Polyset IRIS Twin Steel Container Lunch Pack S...,"[Air-Tight,Spill Proof,Freezer Safe,High quali...",NaN,1256,905.511810
2,1836614,Prime Retail [Anti Glare] [Scratch Proof] Temp...,[Infinix S5 Pro : Specifically designed with H...,PRIME RETAIL Premium GLASS Protectors are the ...,2211,393.700787
3,620873,Joyeux Noel (Litterature & Documents),NaN,NaN,99,433.070000
4,2577345,"ZINGTEL LG Q60 Back Screen Guards, Back Skin, ...",[Value for Purchase : We Value your time & mon...,<p><strong>ZINGTEL</strong> Introduces 3D Carb...,2211,787.401574
5,1650033,OZANGO Bike Exhaust Silencer Wrap Heat Shield ...,[This heat wrap is very durable and tight wove...,OZANGO Silencer Wrap Golden Silencer Wrap Gold...,7393,787.401574
6,920355,Weiler 42006 Tampico Fiber Medium Sweep Floor ...,NaN,NaN,10202,413.000000
7,1378539,Boston Warehouse Artsy Grey Owl Stainless Stee...,[The artsy Grey owl spreaders add whimsy to yo...,NaN,1267,157.400000
8,458720,29 Hours,NaN,NaN,97,598.424000
9,126115,The Making Of The British Landscape: From the ...,NaN,NaN,6328,503.936000


In [ ]:
import pandas as pd



# Concatenate the columns TITLE, DESCRIPTION, and BULLET_POINTS with a space in between and assign the values to a new column named "PRODUCT_LENGTH"
data['text'] = data['TITLE'] + ' ' + data['DESCRIPTION'] + ' ' + data['BULLET_POINTS']

# Print the first 5 rows of the DataFrame to check the new column
print(data.head(5))


   PRODUCT_ID                                              TITLE  \
0     1925202  ArtzFolio Tulip Flowers Blackout Curtain for D...   
1     2673191  Marks & Spencer Girls' Pyjama Sets T86_2561C_N...   
2     2765088  PRIKNIK Horn Red Electric Air Horn Compressor ...   
3     1594019  ALISHAH Women's Cotton Ankle Length Leggings C...   
4      283658  The United Empire Loyalists: A Chronicle of th...   

                                       BULLET_POINTS  \
0  [LUXURIOUS & APPEALING: Beautiful custom-made ...   
1  [Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...   
2  [Loud Dual Tone Trumpet Horn, Compatible With ...   
3  [Made By 95%cotton and 5% Lycra which gives yo...   
4                                                NaN   

                                         DESCRIPTION  PRODUCT_TYPE_ID  \
0                                                NaN             1650   
1                                                NaN             2755   
2  Specifications: Color: Red, Mate

In [ ]:
df = data.copy()
df.drop(['TITLE','DESCRIPTION','BULLET_POINTS'], axis = 1, inplace = True);
df.head(5)

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,text
0,1925202,1650,2125.980000,NaN
1,2673191,2755,393.700000,NaN
2,2765088,7537,748.031495,PRIKNIK Horn Red Electric Air Horn Compressor ...
3,1594019,2996,787.401574,ALISHAH Women's Cotton Ankle Length Leggings C...
4,283658,6112,598.424000,NaN


In [ ]:
df.dropna(inplace=True)
df.isna().sum()

PRODUCT_ID         0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
text               0
dtype: int64

In [ ]:
df.head(5)

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,text
2,2765088,7537,748.031495,PRIKNIK Horn Red Electric Air Horn Compressor ...
3,1594019,2996,787.401574,ALISHAH Women's Cotton Ankle Length Leggings C...
5,2152929,5725,950.000000,HINS Metal Bucket Shape Plant Pot for Indoor &...
7,2026580,6030,984.251967,Delavala Self Adhesive Kitchen Backsplash Wall...
9,2998633,8201,393.700787,Hexwell Essential oil for Home Fragrance Oil A...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_less['TITLE_WITH_ID'])

In [ ]:
y = data_less['PRODUCT_LENGTH']

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
lr.fit(X_train,y_train)

LinearRegression()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
def get_score(y_true,y_pred):
  score = max(0,100*(1-mean_absolute_percentage_error(y_true,y_pred)))
  return score

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
get_score(y_test,y_pred)

0

In [ ]:
lr.score(X_train,y_train)

0.9999999001668014

In [ ]:
lr.score(X_test,y_test)

-0.6855746945318455

###Dropping null values

In [ ]:
data_less_clean = data_less.dropna()

In [ ]:
data_less_clean.head(5)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1748580,MAHARAJA Solenoid Valve for All types of RO wa...,[Easily replaceable by anybody with or without...,"Solenoid Valve for Kent, AquaGuard RO water pu...",1498,400.000000
1,2369493,Indiacase Back Case Cover for i-Phone 12 Pro M...,[Perfect compatible with i-Phone 12 Pro Max. I...,Indiacase Avenger Mask Designer Case That Is N...,12556,551.181102
2,880617,Biona Org Horseradish Mustard 125G,"[As shown in the description,Item condition ne...",Biona org horseradish mustard 125g,2041,236.000000
5,1826480,Platinum Shield Weatherproof Car Cover Compati...,"[FITS: 1985 Dodge Aries Sedan 4 Door,Platinum ...",<b>Platinum Shield Weatherproof Car Cover For ...,7332,2000.000000
6,2643875,"Cocomii Pearl Glitter AirPods 3 Case, Slim Thi...","[UNIQUE, STYLISH, QUALITY: This beautiful case...",<b>COCOMII</b><br><br>We transform amazing ide...,1814,300.000000


In [ ]:
for i in range(2):
  print(data_less_clean['BULLET_POINTS'].iloc[i])

[Easily replaceable by anybody with or without experience. High Quality Solenoid valve.,Compatible with all and any model of all domestic water purifiers,24 Volt DC Solenoid valve]
[Perfect compatible with i-Phone 12 Pro Max. Iron Man design for i-Phone 12 Pro Max phone case looks more stylish and cool with 360 degree Protection.,Comfortable To Touch And Fashion Design Style.It's Simple And Elegance.Avenger Mask Case Specially Made For Luxury Fashionable And Durable Design Which Makes Your Phone Much More Safe.,Attractive Appearance, Feel Warm And Comfortable, To Ensure You Put It Down.,Exquisite Design Style Can Protect Your Phone From Scratches Or Shocks,All Buttons And Jacks Are Accessible Through The Cutouts.,Complete Access To All Features Of The Device Including Microphone Speaker Camera And All Button.]


In [ ]:
y = data_less_clean['PRODUCT_LENGTH']

###Decision Tree

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_less_clean['DESCRIPTION'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
dtr = DecisionTreeRegressor()

In [ ]:
dtr.fit(X_train,y_train)

DecisionTreeRegressor()

In [ ]:
y_pred = dtr.predict(X_test)
get_score(y_test,y_pred)

0

In [ ]:
y_test[:10]

12104     300.000000
5680      314.960630
21098    2910.000000
1178      232.283000
530       393.700787
3736      905.511810
22220    1100.000000
5151      590.551180
11034     472.440944
3919       86.614173
Name: PRODUCT_LENGTH, dtype: float64

In [ ]:
dtr.score(X_train,y_train)

0.9999944618570515

In [ ]:
dtr.score(X_test,y_test)

-1.7058179543853988

### Text cleaning

In [ ]:
import re
def token_temp(text)
    text = re.sub('[^0-9a-zA-Z\/\-\s]', '', text)
    text = text.lower()
    tokens = text.split()
    subtokens = []
    for token in tokens:
        subtokens.extend(re.split('[\/\-]', token))
    tokens = [subtoken for subtoken in subtokens if subtoken]
    return tokens

###BERT

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
data_less_clean.shape

(9173, 6)

In [ ]:
data_less_clean.head(5)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2039862,2924563,GelForce - Electric Gel Ball Blaster - Mini M4...,[RAPID-FIRE GEL BLASTER: Shoot farther and fas...,it s time to have an epic gel ball battle with...,1180,1360.000000
1922458,1523221,Fiskars 194900-1001 Pre-School Training Scisso...,"[Ideal for children ages three and up,#1 teach...",fiskars 194900 1001 pre school training scisso...,11148,730.000000
2062557,938163,"Parachute Army Men Party Favors, 12ct","[12 Plastic Parachute Army Men,Plastic Army Me...",great for pinata filler and loot bags,1111,400.000000
963753,2219489,ZDYWY 12 Inch Alphabet Letter Number Shaped Al...,"[Size: 31 x 18 x 5cm/12.2 x 7.1 x 2inch,Made o...",tasteless non toxic durable not easily defo...,6514,1200.000000
1027831,1985920,AversS Melamine Dinner Set Round Shape Easy to...,[Lightweight & Durable: Melamine dishes is lig...,this dinnerware set made of melamine so you do...,13061,1889.763778


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
data_less_clean.isna().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
dtype: int64

In [ ]:
X = list(data_less_clean["DESCRIPTION"])
y = list(data_less_clean["PRODUCT_LENGTH"])

In [ ]:
data_less_clean["PRODUCT_LENGTH"].info()

<class 'pandas.core.series.Series'>
Int64Index: 9173 entries, 2039862 to 2048322
Series name: PRODUCT_LENGTH
Non-Null Count  Dtype  
--------------  -----  
9173 non-null   float64
dtypes: float64(1)
memory usage: 143.3 KB


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels=None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key:torch.tensor(val[idx]) for key ,val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [ ]:
train_dataset = Dataset(X_train_tokenized,y_train)
test_dataset = Dataset(X_test_tokenized,y_test)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

batch_size = 16

args = TrainingArguments(
    learning_rate=2e-5,
    output_dir = "output",
    per_device_train_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    metric_for_best_model='accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,11321525403.648001


Step,Training Loss
500,11321525403.648001
1000,15240080130.048000


TrainOutput(global_step=1377, training_loss=12331923493.18228, metrics={'train_runtime': 2028.2507, 'train_samples_per_second': 10.854, 'train_steps_per_second': 0.679, 'total_flos': 5792074767525888.0, 'train_loss': 12331923493.18228, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 61135420.0,
 'eval_mse': 61135412.0,
 'eval_rmse': 7818.91357421875,
 'eval_mae': 1533.111328125,
 'eval_r2': -0.03997180282905344,
 'eval_smape': 181.1565735694823,
 'eval_runtime': 60.4744,
 'eval_samples_per_second': 30.343,
 'eval_steps_per_second': 3.803,
 'epoch': 3.0}

In [ ]:
# {'eval_loss': 39074104.0,
#  'eval_mse': 39074104.0,
#  'eval_rmse': 6250.92822265625,
#  'eval_mae': 1594.2906494140625,
#  'eval_r2': -0.06957553840448916,
#  'eval_smape': 198.16600339248433,
#  'eval_runtime': 31.6801,
#  'eval_samples_per_second': 30.24,
#  'eval_steps_per_second': 3.788,
#  'epoch': 3.0}

### Online Tutorial

In [ ]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Amazon/dataset/train.csv")
data_less = data.sample(20000)
data_less_clean = data_less.dropna()

In [ ]:
data_less_clean['DESCRIPTION'] = data_less_clean['DESCRIPTION'].str.lower()

<ipython-input-5-c7612989d1c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_less_clean['DESCRIPTION'] = data_less_clean['DESCRIPTION'].str.lower()


In [ ]:
data_less_clean['DESCRIPTION'] = data_less_clean['DESCRIPTION'].replace('[^a-zA-Z0-9]', ' ', regex = True)

<ipython-input-6-96456d556008>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_less_clean['DESCRIPTION'] = data_less_clean['DESCRIPTION'].replace('[^a-zA-Z0-9]', ' ', regex = True)


In [ ]:
vectorizer = TfidfVectorizer(min_df=5)
X_tfidf = vectorizer.fit_transform(data_less_clean['DESCRIPTION'])
y = data_less_clean['PRODUCT_LENGTH']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_tfidf,y,test_size=0.2)

In [ ]:
clf = Ridge(alpha=1.0, random_state=241)

In [ ]:
clf.fit(X_train, y_train)

Ridge(random_state=241)

In [ ]:
clf.score(X_test,y_test)

-56.22126701638003

In [ ]:
clf.score(X_train,y_train)

0.4302067442751141

## Analysing Data

In [ ]:
import random
def display_points():
  index = random.randint(0,len(data_less_clean)-2)
  bullet_points = data_less_clean.iloc[index]['BULLET_POINTS'][1:-1]
  description = data_less_clean.iloc[index]['DESCRIPTION']
  title = data_less_clean.iloc[index]['TITLE']
  length = data_less_clean.iloc[index]['PRODUCT_LENGTH']
  print(title,'\n')
  print(bullet_points,'\n')
  print(description,'\n')
  print(length,'\n')

In [ ]:
display_points()

Caajib Decor's Designer Pink and Blue Ceramic Planter Pot with Attached Tray for Home, Bedroom, Living Room Indoor & Out Door Gardening, 12 cm (Set of 4) 

🌿【Modern Minimalist Style】It has a sleek style that will compliment any house. This planter can be used for a wide variety of plants like cactus, snake plant, aloe, herbs and succulent, especially suitable for root-depth plants.,🌿【Sturdy Ceramic Material】The plant pot are durable and heavy. You won’t need to worry about it breaking after sun exposure like plastic pots, or rusting in the humid air like metal ones.,🌿【Drainage Hole and Plug】Roots love moisture but are afraid of excessive water. That’s why there’s a drainage hole to keep the plant from getting root rot. It has a removable stopper so you can take it off to drain when you water the plant.,🌿【Finishing Touch to Any Room】This flower pots great for the bedroom, living room, bathroom, dining table, kitchen, windowsill, desktop 

Decorating some green plants in your home will m

In [ ]:
display_points()

Sanuk Women's Yoga Wildlife Flip Flop Sandal,Zebra,7 M US 

Flip-flop featuring cushioned animal-print canvas strap and solid toe post,Textured footbed made from a real yoga mat,Happy U textured outsole 

Unleash the beast and experience the comfort! Women's Sandals Footbed Made From Real Yoga Mat! Happy U Outsole Washed Canvas, Animal Printed Strap with Synthetic Textile Liner Vegan and Vegetarian 

1200.0 



In [ ]:
# product length(inches) / 100 = actual length in inches

In [ ]:
data_less_clean.to_csv("/content/drive/MyDrive/Amazon/dataset/less.csv")

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Amazon/dataset/less.csv")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13911 entries, 0 to 13910
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       13911 non-null  int64  
 1   PRODUCT_ID       13911 non-null  int64  
 2   TITLE            13911 non-null  object 
 3   BULLET_POINTS    13911 non-null  object 
 4   DESCRIPTION      13911 non-null  object 
 5   PRODUCT_TYPE_ID  13911 non-null  int64  
 6   PRODUCT_LENGTH   13911 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 760.9+ KB


In [ ]:
import re
CLEANR = re.compile('<.*?>')
def get_lower(text):
  text = str(text).lower()
  text = re.sub(CLEANR, '', text)
  return text

In [ ]:
data['TITLE'] = data["TITLE"].apply(lambda x : get_lower(x))
data['BULLET_POINTS'] = data["BULLET_POINTS"].apply(lambda x : get_lower(x))
data['DESCRIPTION'] = data["DESCRIPTION"].apply(lambda x : get_lower(x))

In [ ]:
def check_digit(text):
  numbers="0123456789"
  res = False
  for i in text:
      if i in numbers:
          res=True
          break
  return res

In [ ]:
def bullet_points_fixer(text):
  text = text[1:-1]
  send = []
  for line in text.split(','):
    if check_digit(line):
      send.append(line)
  return str(" ".join(send))

In [ ]:
bullet_points_fixer(data["BULLET_POINTS"].iloc[1024])


"rich and full of beauty. they sound lovely so bring this wind chimes home today. out wall of the wind chimes are made of quality thick material so it creates a longer lasting deep tone. only metal wind chimes can be tuned to specific tones and notes.6 heavy aluminum tubes with black matte color.  aluminum tubes range in length from 5.2 inches to 4.7 inches. at the top  the suspension platform is 4.7 inches wide  while the striker is 4.5 inches wide.this wind chime is made of premium quality beech ♬♪outdoor decorative wind chimes： 20'' is the best size for your outdoor and indoor decor. this decorative chime comes with a stronger metal hoo"

In [ ]:
data["PRODUCT_LENGTH"].iloc[1024]

2010.0

In [ ]:
data.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1748580,maharaja solenoid valve for all types of ro wa...,24 volt dc solenoid valve,"solenoid valve for kent, aquaguard ro water pu...",1498,400.000000
1,2369493,indiacase back case cover for i-phone 12 pro m...,perfect compatible with i-phone 12 pro max. ir...,indiacase avenger mask designer case that is n...,12556,551.181102
2,880617,biona org horseradish mustard 125g,package included 1 piece,biona org horseradish mustard 125g,2041,236.000000
3,1826480,platinum shield weatherproof car cover compati...,fits: 1985 dodge aries sedan 4 door weatherpro...,platinum shield weatherproof car cover for 198...,7332,2000.000000
4,2643875,"cocomii pearl glitter airpods 3 case, slim thi...",quality: this beautiful case is compatible wi...,cocomiiwe transform amazing ideas and advanced...,1814,300.000000


In [ ]:
data['BULLET_POINTS'] = data["BULLET_POINTS"].apply(lambda x : bullet_points_fixer(x))

In [ ]:
data.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
data.to_csv("/content/drive/MyDrive/Amazon/dataset/less.csv",index=False)

### After processing

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Amazon/dataset/less.csv")
data.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1748580,maharaja solenoid valve for all types of ro wa...,24 volt dc solenoid valve,"solenoid valve for kent, aquaguard ro water pu...",1498,400.000000
1,2369493,indiacase back case cover for i-phone 12 pro m...,perfect compatible with i-phone 12 pro max. ir...,indiacase avenger mask designer case that is n...,12556,551.181102
2,880617,biona org horseradish mustard 125g,package included 1 piece,biona org horseradish mustard 125g,2041,236.000000
3,1826480,platinum shield weatherproof car cover compati...,fits: 1985 dodge aries sedan 4 door weatherpro...,platinum shield weatherproof car cover for 198...,7332,2000.000000
4,2643875,"cocomii pearl glitter airpods 3 case, slim thi...",quality: this beautiful case is compatible wi...,cocomiiwe transform amazing ideas and advanced...,1814,300.000000


<bound method Series.unique of 0         1498
1        12556
2         2041
3         7332
4         1814
         ...  
13906      348
13907     7978
13908     1407
13909      713
13910     7254
Name: PRODUCT_TYPE_ID, Length: 13911, dtype: int64>